## An Electrode Design Optimisation Example

A design optimisation example loosely based on work by L.D. Couto available at [[1]](https://doi.org/10.1016/j.energy.2022.125966).

The target is to maximise the gravimetric energy density over a range of possible design parameter values, including for example:

cross-sectional area = height x width (only need change one), electrode widths, particle radii, volume fractions and separator width.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP and upgrade dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets -q
%pip install pybop -q

/Users/engs2510/Documents/Git/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


/Users/engs2510/Documents/Git/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


Next, we import the added packages plus any additional dependencies,

In [ ]:
import numpy as np
import pybamm
from pybamm import Parameter

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Let's fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [ ]:
np.random.seed(8)

## Optimising the Parameters

First, we define the model to be used for the parameter optimisation,

In [ ]:
model = pybamm.lithium_ion.SPMe()
pybop.pybamm.add_variable_to_model(model, "Gravimetric energy density [Wh.kg-1]")

parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values.update(
    {
        "Electrolyte density [kg.m-3]": Parameter("Separator density [kg.m-3]"),
        "Negative electrode active material density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Negative electrode carbon-binder density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Positive electrode active material density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
        "Positive electrode carbon-binder density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
        "Cell mass [kg]": pybop.pybamm.cell_mass(),
    },
    check_already_exists=False,
)

Next, we define the model parameters for optimisation. Furthermore, PyBOP provides functionality to define a prior for the parameters. The initial parameters values used in the optimisation will be randomly drawn from the prior distribution.

In [ ]:
parameter_values.update(
    {
        "Positive electrode thickness [m]": pybop.Parameter(
            "Positive electrode thickness [m]",
            prior=pybop.Gaussian(7.56e-05, 0.05e-05),
            bounds=[65e-06, 10e-05],
        ),
        "Positive particle radius [m]": pybop.Parameter(
            "Positive particle radius [m]",
            prior=pybop.Gaussian(5.22e-06, 0.05e-06),
            bounds=[2e-06, 9e-06],
        ),
    }
)

Next, we construct the experiment for design optimisation and the initial state-of-charge,

In [ ]:
experiment = pybamm.Experiment(["Discharge at 1C until 2.5 V (5 seconds period)"])

We can now define the output signal, the problem (which combines the model with the dataset) and construct a cost function which in this example is based on the gravimetric energy density of the cell.

In [ ]:
simulator = pybop.pybamm.Simulator(
    model,
    parameter_values=parameter_values,
    protocol=experiment,
    initial_state={"Initial SoC": 0.7},
    use_formation_concentrations=True,
)
cost = pybop.DesignCost(target="Gravimetric energy density [Wh.kg-1]")
problem = pybop.Problem(simulator, cost)

/var/folders/2s/pn1xwy191_7bv761mlf_cgzh0000gq/T/ipykernel_58043/2045688338.py:1: UserWarning:

The nominal capacity is fixed at the initial model value.



Let's construct PyBOP's optimisation class. This class provides the methods needed to fit the forward model. For this example, we use particle swarm optimisation (PSO). Due to the computational requirements of the design optimisation methods, we limit the number of iterations to 15 for this example.

In [ ]:
options = pybop.PintsOptions(verbose=True, max_iterations=15)
optim = pybop.PSO(problem, options=options)

Finally, we run the optimisation and return the values obtained,

In [ ]:
result = optim.run()
print(f"Initial gravimetric energy density: {result.cost[0]:.2f} Wh.kg-1")
print(f"Optimised gravimetric energy density: {result.best_cost:.2f} Wh.kg-1")

Halt: Maximum number of iterations (15) reached.
OptimisationResult:
  Initial parameters: [7.56456024e-05 5.27456414e-06]
  Optimised parameters: [8.27568156e-05 2.07975193e-06]
  Final cost: 277.8414399457407
  Optimisation time: 32.96286225318909 seconds
  Number of iterations: 15
  SciPy result available: No


Initial gravimetric energy density: 254.85 Wh.kg-1
Optimised gravimetric energy density: 277.84 Wh.kg-1


## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

### Cost Landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:


In [ ]:
result.plot_surface();

### Comparing System Response

We can quickly plot the system's response using the estimated parameters compared to the initial parameters:


In [ ]:
problem.target = "Voltage [V]"
pybop.plot.problem(problem, problem_inputs=result.x, title="Optimised Comparison");